# Milestone 3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

We start by loading the tables.

In [ ]:
df_plot = pd.read_csv("data/MovieSummaries/plot_summaries.txt", sep='\t', header=None, names=["wikiID", "plot"])

df_meta = pd.read_csv("data/MovieSummaries/movie.metadata.tsv", sep='\t', header=None, 
    names=["wikiID", "freeID", "name", "release_date", "revenue", "runtime", "languages", "countries", "genres"])

df_char = pd.read_csv("data/MovieSummaries/character.metadata.tsv", sep='\t',header=None, 
    names=["WikiID", "freeID", "release_date", "char_name", "actor_DOB", "actor_gender", "actor_height", "actor_ethnicity",
           "actor_name", "Actor_age", "freeID_char_map", "FreeID_char", "FreeID_actor"])

df_char_names = pd.read_csv('data/MovieSummaries/name.clusters.txt', sep="\t", header=None, 
    names=["char_name", "freeID_char_map"])

df_tropes = pd.read_csv('data/MovieSummaries/tvtropes.clusters.txt', sep='\t', header=None, names=["trope", "details"])

## Data features and missing values

In [ ]:
df_plot.head()

In [ ]:
df_plot.info()

In [ ]:
df_meta.head()

In [ ]:
df_meta.info()

Looking at the output above, we might think that the languages, countries and genres columns have no missing elements but actually some elements are just empty dictionary values

In [ ]:
print(df_meta['languages'].value_counts()['{}'])
print(df_meta['countries'].value_counts()['{}'])
print(df_meta['genres'].value_counts()['{}'])

Also, if we want to make use of the release date we can already convert it to datetime and check if we have more missing values.

In [ ]:
df_meta['release_date'] = pd.to_datetime(df_meta['release_date'], errors = 'coerce')
print(df_meta['release_date'].shape[0] - df_meta['release_date'].isnull().sum())

In [ ]:
df_char.head()

In [ ]:
df_char.info()

In [ ]:
df_char_names.head()

In [ ]:
df_char_names.info()

In [ ]:
df_tropes.head()

In [ ]:
df_tropes = df_tropes.drop(["details"], axis=1).join(pd.json_normalize(df_tropes["details"].map(json.loads).tolist())).rename(
    columns={"id":"freeID_char_map"})

df_tropes.head()

In [ ]:
df_tropes.info()

---
## NLPCore

After we looked at the Stanford CoreNLP-processed summaries, we assumed it would not be useful for us.

The code below is an example where we load and parse the data to retrieve an element.

In [ ]:
import gzip
import os
from bs4 import BeautifulSoup

path = "data/corenlp_plot_summaries/"
files = os.listdir(path)
print(files[0])
f = gzip.open(path+files[0], 'rb')

test = f.read().decode()
# print(test)

Bs_data = BeautifulSoup(test, "xml")
print(Bs_data.find("sentence", {'id':'20'}).find('NER').text)

---
## Exploratory data analysis

### Genres, languages and countries

We want to identify the most represented genres, languages and countries.

In [ ]:
def fcount(df):
    df2 = pd.DataFrame(data={
        'id': df.apply(lambda x: list(json.loads(x).keys())),
        'name': df.apply(lambda x: list(json.loads(x).values()))
    })
    
    distinctf = []
    counter = []

    for i in range(df2.shape[0]):
        fid = df2["id"].iloc[i]
        fname = df2["name"].iloc[i]
        for j in range(len(fid)):
            if [fid[j], fname[j]] not in distinctf:
                distinctf.append([fid[j], fname[j]])
                counter.append(1)
            else:
                counter[distinctf.index([fid[j], fname[j]])] += 1

    new = pd.DataFrame(data={
        'id': [s[0] for s in distinctf],
        'name': [s[1] for s in distinctf],
        'count': counter
    })
    
    return new.sort_values(by="count", ascending=False, ignore_index=True)

In [ ]:
df_genre = fcount(df_meta["genres"])
df_genre.head(20)

In [ ]:
df_lang = fcount(df_meta["languages"])
df_lang.head(20)

In [ ]:
df_country = fcount(df_meta["countries"])
df_country.head(20)

There might be a correlation between the country and language features.

### Release date distribution

In [ ]:
df_meta.groupby(df_meta["release_date"].dt.year).count()['wikiID'].plot()
plt.show()

We want to study the similarity between two movies so the date of their releases can be interesting. In this case, the exact release date might not be necessary, because it is expected that the difference would lie between movies that have for example 10 years of difference not just a few months. Actually, in some cases we only have the year of the release, so it might be simpler to only store the year of the release for all movies in order to keep homogeneous data.

### Runtime distribution

In [ ]:
print(df_meta['runtime'].max())

2 years of runtime?

In [ ]:
print(df_meta[df_meta['runtime'] >= 400].shape[0])
print(df_meta[df_meta['runtime'] <= 3].shape[0])

More than 6 hours of runtime seems too large and less than 3 minutes seems too short.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_meta[df_meta['runtime'] < 400]['runtime'], bins=400)
ax.set_title('Distribution of runtime')

plt.show()

We can observe that the distribution is more concentrated around a runtime of 90 minutes but it appears there is a cluster around 20 which might corresponds to short films (6th most represented movie genre in the dataset).

The exact value of the runtime does not always seem to be reliable data, and looking at the distribution it might be a good idea to replace the runtime by an attribute like "short", "long" or "don't know" (if runtime is NaN).

### Runtime distribution

In [ ]:
df_meta['revenue'].dropna().hist(bins=100)

In [ ]:
print(df_meta['revenue'].max())

In [ ]:
print(df_meta[df_meta['revenue']==df_meta['revenue'].max()].name)

There is a very large different between most of the movies and the movies that had a huge success such as "Avatar".

Note that the box office revenue is absent for most of the movies and we might not need it to compute the similarity between two movies. Therefore, we estimated that this feature would not be useful for our project.

### Distribution of the number of words per plot

In [ ]:
#use raw plot_summaries and count nb of words in each plot 
df_plot_fltr = df_plot.copy()
df_plot_fltr['nb_words'] = df_plot_fltr['plot'].apply(lambda n: len(n.split()))

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_plot_fltr['nb_words'], bins=1000)
ax.set_title('Distribution of the number of words per plot')

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_plot_fltr['nb_words'], bins=1000)
ax.set_title('Zoom between 0 and 100 words')
plt.xlim([0, 100])

plt.show()

We still find some extreme values in the distribution. In particular, some plots have less than 20 words, on the other hand there are plots with thousands of words.

In the preprocessing, we decided to discard the movies with plots that are less than 10 words as they might not be relevant when we will apply topic extraction.

### Distribution of the number of characters per movie

First we will group all the characters by movie. So if a movie has at least one labeled character, we will be able to count its number of character so that we can have an insight on the distribution of the number of characters for a movie.

In [ ]:
df_char_fltr = df_char[['freeID','FreeID_actor']].copy()
df_char_fltr = df_char_fltr.groupby(['freeID']).size()

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_char_fltr, bins=120)
ax.set_title('Distribution of the number of labeled characters per movie')

plt.show()

In [ ]:
print("Number of movies without any labeled character: ", 
      df_meta.shape[0] - df_meta[df_meta['freeID'].isin(df_char_fltr.index)].shape[0])

This can be missing information but it is also logical that some movies such as documentaries do not have any character.

In [ ]:
red_square = dict(markerfacecolor='r', marker='s')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.boxplot(df_char_fltr, vert=False, flierprops=red_square)
ax.set_title('Boxplot for the number of labeled characters per movie')
plt.show()

In [ ]:
print("1rst quartile, median and 3rd quartile values: ")
print(df_char_fltr.quantile([0.25,0.5,0.75]))

## Join the dataframes

The feature connecting dataframes together is the Wikipedia ID. Also there are more metadatas of movies (81741 movies) than plots (42303 movies). We will only keep the metadatas of the movies we know the plot of.

In [ ]:
dfull = df_meta.merge(df_plot, how='inner', on="wikiID")
dfull.head()

Only a small fraction of the characters have been labellised with a trope (500), compared to the number of unlabeled (450669). Here is the dataframe containing the characters with trope

In [ ]:
df_char_trope = df_char.merge(df_tropes, how='inner', on='freeID_char_map')

df_char_trope.drop(columns=["char", "actor"], inplace=True)
df_char_trope.head()

In [ ]:
df_char.info()

In [ ]:
df_char_list = df_char[['freeID','char_name']].groupby('freeID')['char_name'].apply(list).reset_index(name='characters')
df_char_list.info()

In [ ]:
df_actor_list = df_char[['freeID','actor_name']].groupby('freeID')['actor_name'].apply(list).reset_index(name='actors')
df_actor_list.info()

---
## Preprocessing

We decided to filter our data, and remove the elements without a genre or with less than 10 words in their plot, because these are the features that we think are the most important in the idea of our project.

In [ ]:
df_clean = (dfull[(dfull['genres'] != '{}') & (dfull['plot'].apply(lambda s: len(s.split())) > 10)]).drop(
    columns=['revenue'])

print('number of movies remaining after movie metadata preprocessing: ', df_clean.shape[0])

We are formatting the data to facilitate the analysis.

In [ ]:
df_clean['countries'] = pd.DataFrame(data={'countries': df_clean['countries'].apply(lambda x: set(json.loads(x).values()))})
df_clean['languages'] = pd.DataFrame(data={'languages': df_clean['languages'].apply(lambda x: set(json.loads(x).values()))})
df_clean['genres'] = pd.DataFrame(data={'genres': df_clean['genres'].apply(lambda x: set(json.loads(x).values()))}) 

We decided to replace the full release date by the year of release, so that all the movies have only an integer for this feature, and because we estimated that the year was sufficient to establish a similarity on the release date. 

In [ ]:
df_clean['release_date'] = df_clean['release_date'].dt.year.apply(lambda x: int(x) if x == x else None)

We decided to redefine the runtime either as "short" for movies with a runtime less than 40 minutes (a short film is generally less than 40 minutes), "long" for movies with a runtime larger than 40 minutes or "unknown" if we do not know the runtime.

In [ ]:
df_clean['runtime'] = df_clean['runtime'].apply(lambda x: 'short' if x <= 40 else ('long' if x > 40 else None))

We add the actors list and character list to the df_clean dataframe

In [ ]:
df_clean = df_clean.merge(df_char_list, how='inner', on='freeID')
df_clean = df_clean.merge(df_actor_list, how='inner', on='freeID')

In [ ]:
df_clean['actors'] =pd.DataFrame(data={'actors': df_clean['actors'].apply(lambda x: list(x))})
df_clean['actors'] =pd.DataFrame(data={'actors': df_clean['actors'].apply(lambda x: [item for item in x if not(pd.isna(item)) == True] )})
df_clean['actors'] =pd.DataFrame(data={'actors': df_clean['actors'].apply(lambda x: set(x) )})
df_clean['characters'] =pd.DataFrame(data={'characters': df_clean['characters'].apply(lambda x: list(x))})
df_clean['characters'] =pd.DataFrame(data={'characters': df_clean['characters'].apply(lambda x: [item for item in x if not(pd.isna(item)) == True] )})
df_clean['characters'] =pd.DataFrame(data={'characters': df_clean['characters'].apply(lambda x: set(x))})

In [ ]:
df_clean.head(20)

In [ ]:
import math
print(df_clean['characters'][1])
print(pd.isna(df_clean['characters'][1]))

---
## Scraping

We already performed test to scrape data with the code below.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pywikibot
from pywikibot import *

def scraping(wiki_ID):
    list = []

    query_base = "https://query.wikidata.org/bigdata/namespace/wdq/sparql?format=json&query="

    wiki_ID_encoded = wiki_ID.replace("/","%2F")

    pre_str = ( "https://query.wikidata.org/bigdata/namespace/wdq/sparql?format=json&query=PREFIX%20wd%3A%20%3Chttp%3A%2F%2" +
        "Fwww.wikidata.org%2Fentity%2F%3E%0APREFIX%20wdt%3A%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0APREF" +
        "IX%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0ASELECT%20%20%3Fs%20%3FsLabel%20%3Fp%20%20%3Fo%2" +
        "0%3FoLabel%20WHERE%20%7B%0A%20%3Fs%20wdt%3AP646%20%22" )
    post_str = ( "%22%20%0A%0A%20%20%20SERVICE%20wikibase%3Alabel%20%7B%0A%20%20%20%20bd%3AserviceParam%20wikibase%3Alangua" +
        "ge%20%22en%22%20.%0A%20%20%20%7D%0A%20%7D" )

    query = pre_str + wiki_ID_encoded + post_str

    response = requests.get(query)
    soup = BeautifulSoup(response.content, "html.parser")

    results = soup.find(id="results")

    try:
        entity_wiki_id = response.json()['results']['bindings'][0]['s']['value']
    except:
        print("Scraping failed")
        return [None,None]
        

    entity_wiki_id = response.json()['results']['bindings'][0]['s']['value']

    str = entity_wiki_id.split('/')
    entity = str[-1]
    print(entity)
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    item = pywikibot.ItemPage(repo, entity)
    item_dict = item.get()
    clm_dict = item_dict["claims"] # Get the claim dictionary

    #P57 for a movie director
    try:
        clm_list = clm_dict["P57"]
        for clm in clm_list:
                ...
                clm_trgt = clm.getTarget()
        list.append(clm_trgt.labels['en'])
    except:
        list.append(None)

    #P462 for the color
    try:
        clm_list = clm_dict["P462"]
        for clm in clm_list:
                ...
                clm_trgt = clm.getTarget()
        list.append(clm_trgt.labels['en'])
    except:
        list.append(None)

    print('scraped sucessfuly')
    return list

In [ ]:
sub_df = df_clean

list2 = [[]]
for index, row in sub_df.iterrows():
    list2.append(scraping(row['freeID']))

In [ ]:
df_scrapped = pd.DataFrame (list2, columns = ['director', 'color'])

In [ ]:
df_scrapped.to_csv('scrapped.csv', encoding='utf-8')

In [ ]:
df_scrapped = pd.read_csv('scrapped.csv')
df_scrapped.drop(columns = df_scrapped.columns[0], axis = 1, inplace= True)
df_scrapped.head()

In [ ]:
df_clean = df_clean.join(df_scrapped)
df_clean.head()

---
## Graph construction

The aim of the project is to build a weighted graph connecting all movies together. There are two ways to represent a graph:
- Adjacency Matrix
- Adjacency List

However, as calculated in the README, storing the Adjacency Matrix will be about 3.6 GB if we assume each weight is a float. We believe this file is too large, so we will represent it as an adjacency list, and for each vertex we will store the top 100 (for example) most similar movies. This implies that the graph will be directed. Also, we take more than the number of movies we will recommend directly because we want to be able to apply some filters in the recommandation tool (by language for example) without having to recompute the whole graph. For the compute time, we will have approximately 42'306×42'305÷2 = 894'877'665 comparisons to make, which we're confident we'll be able to compute in a reasonable time.

---
## Choice of attributes based on data exploratory analysis

We try to have a maximum of useful attributes to estimate the similarity between the films of the dataset, and to keep as much movies as possible to maximize the chance of being able to recommand similar movies.

When we "filter" an attribute, we remove all instances that are missing that attribute's information.

We believe that the mandatory attribute conditions to be retained in our final dataset are:
* A present title: without a film name, there is no film (hard to name it, to advise it)
* At least one genre labeled (probably one of the most significant attributes for similarity)
* A plot of at least 10 existing words (impose a minimum size so that the subsequent analysis of the topics is accurate)

We eliminate the 'revenue' attribute, due to the small number of movies described by this attribute.

The attributes that are not filtered, but kept to help with the information they carry are:
* Runtime
* Release date
* Language: will be ignored if not provided
* Country: will be ignored if not provided
* Characters: Will be ignored if not provided.

By exploring the wikidata pages by scraping, we recover the name of the producer and the information on the color of the film. These two attributes
were the most constantly present among other more scattered ones (filming location, cinematographer, composer, etc.). We therefore create an additional dataset thanks to the code present in the "scraping" section above.

We discretize the `date_release` in years of publication. The runtime is transformed into short/long categories.

Among the attributes selected for further analysis, we can create attributes resulting from a multiplication between two attributes, such as color and release date.

We have these attributes, with their possible values if categorized in `df_clean`:
- `Plot`
- `Title`
- `Release_date`
- `Genres`: genres list
- `Countries`: countries list
- `Language`: languages list
- `Runtime`: `short`, `long` or `don't know`
- `Director`: Director name or `None`
- `Color`: colored, black and white or `None`

In `df_clean_char`, we holded only the actors names and character names.

---
## Methods to study similarities accross chosen attributes

### 1. Computing the global similarity between movies taking into account all attributes


In the following parts, we will see methods to compute the respective similarity between movies based on each respective feature. We will want to merge all these similarities into one global similarity metric that will take into accounts all attributes.
We compute the total similarity between two films as followed:
$$
S_{tot}=\sum_{i}^{N} w_iS_i
$$
where $w_i$ is the weight associated to the similarity $S_i \in [0,1]$ of the attribute $i \in $ {plot, title, genre, runtime, release_year, country, language, actors, characters, director, color }.

### 2. Algorithm to study similarity between movies attributes

In this part we will propose several methods in order to assess the similarity between movies based on the attributes that we chose here-above. In the [Annexe: Mathematical definitions related to similarity](#Annexe:-Mathematical-definitions-related-to-similarity) we explain the mathematical details and definitions on which the following algorithms are based.

In [ ]:
import math
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import gensim

#### Jaccard similarity implementation
**Method**
The jaccard similarity is more appropriate to compute similarities between datasets of categorical attributes where we want to see if these datasets have some attributes in common or not, and how many they are.

**Implementation**
`jaccard_similarity` function written here below. Since we compare to what extent different movies attributes contain the exact same words, we may need to add a lemmatizer for example to take care of same words being in the singular or plural form, or having uppercase or lowercase initial letter.

**Movie Attribute similarity that can be studied**
- Movie genre
- Movie Language 
- Movie Country
- Actors
- Characters
- Director

Here we can use this method for these attributes because we want to find which of those attributes contain the exact same words and study the similarity based on this analysis. 

In [ ]:
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two sets x and y"""
  intersection_cardinality = len(set.intersection(*[x, y]))
  union_cardinality = len(set.union(*[x,y]))
  if union_cardinality == 0:
    return 0.0
  else:
    return intersection_cardinality/float(union_cardinality)


list1 = ['Drama', 'Mystery', 'Fantasy', 'Adventure']
list2 = ['Fantasy Adventure', 'Adventure', 'Epic', 'Fantasy', 'Drama', 'Action', 'Sword and sorcery films']
list3= ['Satire', 'Comedy', 'Family film']
list4= ['Musical', 'Drama', 'Comedy']
list5= ['Romantic comedy', 'Romance Film', 'Comedy']
list6= ['Comedy']
list7=[]
list8= ['Action']
list9=[]

print("Example of similarities computed with jaccard similarity for the attribute -Movie genre- :")
print("Between lists 1 and 2:",jaccard_similarity(set(list1), set(list2)))
print("Between lists 3 and 4:",jaccard_similarity(set(list3), set(list4)))
print("Between lists 5 and 6:",jaccard_similarity(set(list5), set(list6)))
print("Between lists 3 and 6:",jaccard_similarity(set(list3), set(list6)))
print("Between lists 3 and 7:",jaccard_similarity(set(list3), set(list7)))
print("Between lists 2 and 8:",jaccard_similarity(set(list2), set(list8)))
print("Between lists 7 and 9:",jaccard_similarity(set(list7), set(list9)))
print("Between lists 6 and 8:",jaccard_similarity(set(list6), set(list8)))
print("Between lists 8 and 8:",jaccard_similarity(set(list8), set(list8)))

#### Similarity between binary attributes
**Method**

We have a unique discrete attribute that can take the values: $\left\{1, 0, \text{Nan} \right\}$, where NaN is the case where we don't have information about this attribute.  

**Implementation**

The similarity here is simply 1 if both attributes are equal to 1 or both equal to 0. If at least one of the two attributes is Nan then the similarity is 0.

**Movie Attribute similarity that can be studied**

- Color (1: Movie in colours, 0: Movie in black and white, NaN)
- Runtime (1: full-length film, 0: short film, NaN)

In [ ]:
def isNaN(string):
    return string != string

def binary_similarity(x,y):
    '''
    returns the similarity in values of 
    0 or 1
    '''
    if(isNaN(x) | isNaN(y)):
        return 0
    if(x==y):
        return 1
    else:
        return 0   

#### Similarity between positive integers attributes
**Method**
We want compute the similiarity between two one-dimension continuous variables that belong to $\mathbb{N}$. 

**Implementation**
We start by computing the distance between the two positive integer by computing the manhattan distance: 
$$
d(A,B)=|A-B|
$$
We want to normalize this distance so that $d \in [0,1]$. We do a Min-Max normalization that performs a linear transformation on the original data. This technique gets all the scaled data in the range $[0,1]$. This allows to preserve the relationships among the original data values. The formula to achieve this is the following:
$$
d_{scaled}=\frac{d-d_{min}}{d_{max}-d_{min}}
$$
The cost of having this bounded range is that we will end up with smaller standard deviations, which can suppress the effect of outliers.
We can finally compute the similarity based on this distance:
$$
S=1-d_{scaled}
$$
where $S \in [0,1]$.

**Movie Attribute similarity that can be studied**
- Release year

In [ ]:
def integer_similarity(distance_matrix):
    '''
    inputs: matrice of distances
    output: similarity associated to normalized manhattan distance
    '''

    integ_similiarity=np.zeros(distance_matrix.shape)
    
    idx=np.where(~np.isnan(distance_matrix))
    idx2=np.where(np.isnan(distance_matrix))

    d_max = np.max(distance_matrix[idx])
    d_min = np.min(distance_matrix[idx])
    
    integ_similiarity[idx]=np.ones(distance_matrix[idx].shape) - (distance_matrix[idx]-d_min)/(d_max-d_min)    
    integ_similiarity[idx2]=0
    
    return integ_similiarity

#### tf-idf : Term frequency-inverse document frequency
**Term frequency and Inverse document frequency**

$$
tf(t,d)=\frac{f_{t,d}}{ \sum_{t' \in d} f_{t',d} } \quad \textrm{and} \quad idf(t,D)=log(\frac{N}{|d\in D:t\in d|})
$$
where $f_{t,d}$ = raw frequency = number of times a term $t$ occurs in document $d$, $ \sum_{t' \in d} f_{t',d}$=total number of terms $t'$ in $d$ by suming each independent occurrence
<br>
where $N$=$|D|$= number of documents in corpus $D$, $|d\in D:t\in d|$= number of documents where t appears (where $tf(t,d)\neq 0$)

To prevent bias towards longer documents, the term frequency can be computed as follow:
$$
tf(t,d)=0.5+0.5\frac{f_{t,d}}{max(f_{t',d}:t'\in d)}
$$
To avoid divinding by zero, the idf denominator can ba adjusted as follow:
$$
idf(t,D)=log(\frac{N}{1+|d\in D:t\in d|})
$$
**Term frequency-inverse document frequency**

From the term frequency and the inverse document frequency we can compute the tf-idf:
$$
tf-idf(t,D)=tf(t,d) \cdot idf(t,D)
$$

The tf-idf reflects how important a word is to a document in corpus. tf-idf increase proportionally with the number of times a word appear in a document, but it is offset by the number of documents in the corpus that contains the word. It is high when there is high term frequency in document d **and** low document frequency of the term in the whole corpus. This helps to adjust the fact that some words appear more frequently that we can define as common terms. It tends to filter out those common words.

**Implementation**

tfidf can be implemented using the `sklearn` or `GenSim` opensource library. After obtaining tf-idf associated for each word in each text, we can associate to each text a vector composed of his words tf-idf and null values for other words.
Then we can compute the **cosine similarity** between those texts represented as vectors.

**Movie Attribute similarity that can be studied**
- Title

In [ ]:
def dot(A,B): 
  return (sum(a*b for a,b in zip(A,B)))

def cosine_similarity_simple(a,b):
  """ returns cosine similarity between two lists """
  return dot(a,b)/((dot(a,a)**.5)*(dot(b,b)**.5)) 

print("Example cosine similarity between vectors [1,0] and [1,1]: ", cosine_similarity_simple([1,0],[1,1]))

In [ ]:
from re import sub
from sklearn.metrics.pairwise import cosine_similarity
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity



In [ ]:
from collections import Counter
#df_clean["name"].head()
Counter(" ".join(df_clean["name"]).split()).most_common(100)

In [ ]:
stopwords_headlines=['the','of','the','and','in','a','to','my','for','on','&','from']

In [ ]:
def headlines_similarity(df_headlines, list_stopwords):
    
    documents = list(df_headlines)
    vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=True,stop_words=list_stopwords)
    X = vectorizer.fit_transform(documents)
    list_vectors = X.toarray()
    headlines_similarity=sklearn.metrics.pairwise.cosine_similarity(list_vectors)
    
    return headlines_similarity
    

In [ ]:
list1= 'Harry Potter and the Chamber of Secrets'
list2= 'Gnomes and Trolls: The Secret Chamber'
list3= 'Harry Potter and the Half-Blood Prince'
list4= 'The Taking of Prince Harry'
list5= 'The Prince and Me'
list6= 'Despicable Me'
list7= 'Just Me and You'
list8= 'The Secret Life of Bees'
headlines = [list1,list2,list3,list4,list5,list6,list7,list8]

def create_heatmap(similarity, labels, cmap = "YlGnBu"):
  df = pd.DataFrame(similarity)
  df.columns = labels
  df.index = labels
  fig, ax = plt.subplots()
  sns.heatmap(df, cmap=cmap)

testsimilarity = headlines_similarity(headlines, stopwords_headlines)
print("Example cosine similarities between 8 movie titles: \n", testsimilarity)

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(testsimilarity[0])[::-1]
print("Similarities computed between the title 'Harry Potter and the Chamber of Secrets' and the other titles:")
for idx in sorted_indexes:
    print(idx,'\t', testsimilarity[0,idx],'\t', headlines[idx])
  
create_heatmap(testsimilarity, headlines)

#### GloVe algorithm
**Definition**
The `GloVe` algorithm is an opensource standford algorithm that associates similarity between words by obtaining vectors representations for words and by mapping them into a meaningful space.

**Implementation** 
We can apply soft cosine similarity computation to study similarity between texts. The matrix S from the soft cosine definition can be calculated with the library `GenSim` and using the `GloVe` algorithm. We would obtain a dataset containing a vector for all the words in our corpus and we could compute the matrix S associating a similarity matrix to all of these words. Then with the soft cosine similarity we could analyse the similarity between the documents in our corpus. 

**Movie Attribute similarity that can be studied**
- Title

In [ ]:
from re import sub
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
from gensim.models.word2vec import Word2Vec

In [ ]:
from nltk.corpus import stopwords
from nltk import download
download("stopwords")  # Download stopwords list.

# Load the model: this is a big file, can take a while to download and open
glove = api.load('glove-wiki-gigaword-50') 
#glove = api.load('text8')  # download the corpus and return it opened as an iterable

In [ ]:
def preprocess(doc, stopwords):
    # From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

def compute_similarity_index(model_name):
    model = api.load(model_name)     
    similarity_index = WordEmbeddingSimilarityIndex(model)
    return similarity_index

def headlines_soft_similarity(df_headlines, stopwords, similarity_index):
    # From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
    
    documents = list(df_headlines)
    headlines_similarity=np.zeros([len(documents),len(documents)])

    # Preprocess the documents
    corpus = [preprocess(document,stopwords) for document in documents]
    dictionary = Dictionary(corpus)
    tfidf = TfidfModel(dictionary=dictionary)
    # Create the term similarity matrix.  
    similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

    for i in range(len(documents)):
        query_tf = tfidf[dictionary.doc2bow(corpus[i])]
        index = SoftCosineSimilarity(tfidf[[dictionary.doc2bow(document) for document in corpus]],similarity_matrix)
        headlines_similarity[i,:] = index[query_tf]

    return headlines_similarity

In [ ]:
stopwords_headlines=['the','of','the','and','in','a','to','my','for','on','&','from']
similarity_index = compute_similarity_index('glove-wiki-gigaword-50') # or 'text8'

In [ ]:
#df_headlines = df_clean['name']
list1= 'Harry Potter and the Chamber of Secrets'
list2= 'Gnomes and Trolls: The Secret Chamber'
list3= 'Harry Potter and the Half-Blood Prince'
list4= 'The Taking of Prince Harry'
list5= 'The Prince and Me'
list6= 'Despicable Me'
list7= 'Just Me and You'
list8= 'The Secret Life of Bees'
headlines = [list1, list2,list3,list4,list5,list6,list7,list8]

stopwords1=[]
test_without_stopwords = headlines_soft_similarity(headlines, stopwords1, similarity_index)

stopwords2=stopwords_headlines
test_few_stopwords = headlines_soft_similarity(headlines, stopwords2, similarity_index)

stopwords3 = set(stopwords.words("english"))
test_full_stopwords = headlines_soft_similarity(headlines, stopwords3, similarity_index)

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(test_without_stopwords[0])[::-1]
print("Soft similarities computed between the title 'Harry Potter and the Chamber of Secrets' and the other titles:")
for idx in sorted_indexes:
    print(idx,'\t', test_without_stopwords[0,idx],'\t',headlines[idx])

create_heatmap(test_without_stopwords,headlines)

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(test_few_stopwords[0])[::-1]
print("Soft similarities computed between the title 'Harry Potter and the Chamber of Secrets' and the other titles:")
for idx in sorted_indexes:
    print(idx,'\t', test_few_stopwords[0,idx],'\t',headlines[idx])

create_heatmap(test_few_stopwords,headlines)

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(test_full_stopwords[0])[::-1]
print("Soft similarities computed between the title 'Harry Potter and the Chamber of Secrets' and the other titles:")
for idx in sorted_indexes:
    print(idx,'\t', test_full_stopwords[0,idx],'\t',headlines[idx])

create_heatmap(test_full_stopwords,headlines)

In [ ]:
df_clean.info()

In [ ]:
df_clean.head(10)

In [ ]:
#from nltk.corpus import stopwords
#from nltk import download
#download("stopwords")  # Download stopwords list.

stopwords_headlines=['the','of','the','and','in','a','to','my','for','on','&','from']
similarity_index = compute_similarity_index('glove-wiki-gigaword-50')
#similarity_index = compute_similarity_index('glove-wiki-gigaword-100')
#similarity_index = compute_similarity_index('text8')

In [ ]:
#create similarity matrices for each similarity matrice

def similarity_fct(df):
    '''jacc_genre = np.zeros((df.shape[0], df.shape[0]))
    jacc_language = np.zeros((df.shape[0], df.shape[0]))
    jacc_country = np.zeros((df.shape[0], df.shape[0]))
    jacc_actors = np.zeros((df.shape[0], df.shape[0])).astype('uint16')
    jacc_characters = np.zeros((df.shape[0], df.shape[0])).astype('uint16')
    bin_director = np.zeros((df.shape[0], df.shape[0]))
    bin_color = np.zeros((df.shape[0], df.shape[0]))
    bin_runtime = np.zeros((df.shape[0], df.shape[0]))
    integ_date = np.zeros((df.shape[0], df.shape[0]))
    cosine_headlines = np.zeros((df.shape[0], df.shape[0])).astype('uint16')'''
    #integ_date = np.zeros((df.shape[0], df.shape[0])).astype('uint16')
    idx_nan = np.isnan(df["release_date"].to_numpy())
    all_idx = np.repeat(True, df.shape[0])
    print("idenxes done")
    diff_date = np.tile(df["release_date"].to_numpy().astype('int16'), (df.shape[0],1))
    print('stockage_created')
    diff_date = diff_date - diff_date.transpose()
    diff_date = np.abs(diff_date)
    print('difference calculated')
    diff_date[np.ix_(idx_nan, all_idx)] = -1
    diff_date[np.ix_(all_idx, idx_nan)] = -1
    print('replaced nan')
    #diff_date[idx_nan][:] = -1
    #print(diff_date)
    #idx_no_nan = np.where(~np.isnan(diff_date))
    #print('created idx no_nan')
    #idx_nan    = np.where( np.isnan(diff_date))
    #print('created idx nan')

    diff_date_max = np.max(diff_date)

    print(diff_date_max)
    
    diff_date[np.ix_(idx_nan, all_idx)] = diff_date_max
    diff_date[np.ix_(all_idx, idx_nan)] = diff_date_max
    
    print('replaced nan')
    
    integ_date = np.zeros((df.shape[0], df.shape[0]), dtype = 'uint16')
    
    print('stockage_created 2')
    
    integ_date = 65535 - (diff_date*65535/diff_date_max).astype('uint16')
    np.fill_diagonal(integ_date, 65535)
    #soft_cosine_headlines = np.zeros((df.shape[0], df.shape[0]))
    ID_index = []

    integ_date.tofile("integ_date")
    matrices_dict = {}
    

    '''for i in range(len(df)):
        #print('i',i)
        for j in range(i+1,len(df)):
            #fill only the half of the similarity matrices
            jacc_characters[i,j] = jaccard_similarity(df['characters'].iloc[i],df['characters'].iloc[j]) * 65535
        print(i)
    jacc_characters += jacc_characters.transpose()
    np.fill_diagonal(jacc_characters, 65535)
    jacc_characters.tofile("jacc_characters")
    print("jacc_characters done")
    
    for i in range(len(df)):
        #print('i',i)
        for j in range(i+1,len(df)):
            #fill only the half of the similarity matrices
            jacc_actors[i,j] = jaccard_similarity(df['actors'].iloc[i],df['actors'].iloc[j]) * 65535
        print(i)
    jacc_actors += jacc_actors.transpose()
    np.fill_diagonal(jacc_actors, 65535)
    jacc_actors.tofile("jacc_actors")
    print("jacc_actors done")'''
    
    #cosine_headlines = (headlines_similarity(df['name'], stopwords_headlines) * 65535).astype('uint16')
    #cosine_headlines.tofile("cosine_headlines")
    #print("cosine_headlines done")
    
    '''for i in range(len(df)):
        ID_index.append(df['name'][i])
        #print('i',i)
        for j in range(len(df)):
            #fill only the half of the similarity matrices
            if(i<j):
                jacc_genre[i,j] = jaccard_similarity(df['genres'].iloc[i],df['genres'].iloc[j])
                jacc_language[i,j] = jaccard_similarity(df['languages'].iloc[i],df['languages'].iloc[j])
                jacc_country[i,j] = jaccard_similarity(df['countries'].iloc[i],df['countries'].iloc[j])
                jacc_actors[i,j] = jaccard_similarity(df['actors'].iloc[i],df['actors'].iloc[j])
                jacc_characters[i,j] = jaccard_similarity(df['characters'].iloc[i],df['characters'].iloc[j])
                bin_director[i,j] = binary_similarity(df['director'].iloc[i],df['director'].iloc[j])
                bin_color[i,j]= binary_similarity(df['color'].iloc[i],df['color'].iloc[j])
                bin_runtime[i,j]= binary_similarity(df['runtime'].iloc[i],df['runtime'].iloc[j])

                if( np.isnan(df['release_date'].iloc[i]) | np.isnan(df['release_date'].iloc[j]) ):
                    integ_date[i,j] = None
                else:
                    integ_date[i,j]=abs(df['release_date'].iloc[i]-df['release_date'].iloc[j])

    jacc_genre = jacc_genre+jacc_genre.transpose()
    np.fill_diagonal(jacc_genre, 1)

    jacc_language = jacc_language+jacc_language.transpose()
    np.fill_diagonal(jacc_language, 1)

    jacc_country = jacc_country+jacc_country.transpose()
    np.fill_diagonal(jacc_country, 1)  

    jacc_actors += jacc_actors.transpose()
    np.fill_diagonal(jacc_actors, 1)

    jacc_characters += jacc_characters.transpose()
    np.fill_diagonal(jacc_characters, 1)

    bin_director += bin_director.transpose()
    np.fill_diagonal(bin_director, 1)

    bin_color += bin_color.transpose()
    np.fill_diagonal(bin_color, 1)

    bin_runtime += bin_runtime.transpose()
    np.fill_diagonal(bin_runtime, 1)
    
    integ_date = integ_date+ integ_date.transpose()
    integ_date = integer_similarity(integ_date)    
    np.fill_diagonal(integ_date, 1)

    cosine_headlines = headlines_similarity(df['name'], stopwords_headlines)
    soft_cosine_headlines = headlines_soft_similarity(df['name'], stopwords_headlines, similarity_index)

    matrices_dict['genre'] = jacc_genre
    matrices_dict['language'] = jacc_language
    matrices_dict['countries'] = jacc_country
    matrices_dict['date'] = integ_date
    matrices_dict['actors'] = jacc_actors
    matrices_dict['characters'] = jacc_characters
    matrices_dict['director'] = bin_director
    matrices_dict['color'] = bin_color
    matrices_dict['runtime'] = bin_runtime
    matrices_dict['headlines'] = cosine_headlines
    matrices_dict['headlines_soft'] = soft_cosine_headlines'''
    matrices_dict['date'] = integ_date

    return matrices_dict, ID_index

In [ ]:
#sub_df = df_clean.loc[0:7]
sub_df = df_clean
sub_df.reset_index(drop=True,inplace=True)
sub_df.head(10)
sim_dict, ID_vect =similarity_fct(df=sub_df)

In [ ]:
sim_dict['date']

In [ ]:
upper_sim = np.triu(sim_dict['date'], 1)
print("upper_sim")
#indices = np.transpose(np.nonzero(upper_sim))
largest_edges = upper_sim*(upper_sim>=65000)
indices = np.transpose(np.nonzero(largest_edges))

In [ ]:
test_idx = 20
i = 0
print(len(indices))
for test_idx in range(len(indices)):
    #if(len(df_clean["characters"].iloc[indices[test_idx][0]]) >= 3 and 
    #   len(df_clean["characters"].iloc[indices[test_idx][1]]) >= 3 ):
    print(indices[test_idx])
    print(df_clean["name"].iloc[indices[test_idx][0]], df_clean["release_date"].iloc[indices[test_idx][0]])
    print(df_clean["name"].iloc[indices[test_idx][1]], df_clean["release_date"].iloc[indices[test_idx][1]])
    i += 1
print(i)

In [ ]:
sub_df2 = df_clean.loc[0:20]
sub_df2.reset_index(drop=True,inplace=True)
sub_df2.head(10)
sim_dict2, ID_vect2 =similarity_fct(df=sub_df2)

On veut pas de NAn dans les similarity matrixes

In [ ]:
for index, (key, value) in enumerate(sim_dict.items()):
    print('NaN count for',key,'similarity matrix is', np.count_nonzero(np.isnan(value)))
    #print(value.shape)
    print(np.max(value))

In [ ]:
for index, (key, value) in enumerate(sim_dict2.items()):
    print('NaN count for',key,'similarity matrix is', np.count_nonzero(np.isnan(value)))
    #print(value.shape)
    print(np.max(value))

Similarity matrix te une seule similarité (ici le genre)

In [ ]:
import matplotlib.pyplot as plt

#matrices_dict['genre'] = jacc_genre
#matrices_dict['language'] = jacc_language
#matrices_dict['countries'] = jacc_country
#matrices_dict['date'] = integ_date
#matrices_dict['actors'] = jacc_actors
#matrices_dict['characters'] = jacc_characters
#matrices_dict['director'] = bin_director
#matrices_dict['color'] = bin_color
#matrices_dict['runtime'] = bin_runtime
#matrices_dict['headlines'] = cosine_headlines
#matrices_dict['headlines_soft'] = soft_cosine_headlines

fig, ax = plt.subplots(figsize=(20,20))
cax = ax.matshow(sim_dict2['headlines'] , interpolation='nearest')
ax.grid(True)
plt.title('Film Similarity matrix')
plt.xticks(range(len(sub_df2)), ID_vect2, rotation=90)
plt.yticks(range(len(sub_df2)), ID_vect2)
fig.colorbar(cax) #ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1]
plt.show()

In [ ]:
import matplotlib.pyplot as plt
  
fig, ax = plt.subplots(figsize=(20,20))
cax = ax.matshow(sim_dict['headlines_soft'], interpolation='nearest')
ax.grid(True)
plt.title('Film Similarity matrix')
plt.xticks(range(len(sub_df)), ID_vect, rotation=90)
plt.yticks(range(len(sub_df)), ID_vect)
fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1])
plt.show()

Similarity matrix with weight (not yet normalized, not all similarites yet)

In [ ]:
print(len(sim_dict))
#weighted similarity fct
w1 = 1
w2 = 1
w3 = 1
w4 = 1
w5 = 1
w6 = 1
w7 = 1
w8 = 1
w9 = 1
w10 = 1
w = [w1,w2,w3,w4,w5,w6,w7,w8,w9]# w10] added when added distance
w_norm = w / np.sum(w)
print(w_norm)
sim_matrix = np.zeros(sim_dict['language'].shape)

for index, (key, value) in enumerate(sim_dict.items()):
    sim_matrix = sim_matrix + w_norm[index]*value


In [ ]:
import matplotlib.pyplot as plt
  
fig, ax = plt.subplots(figsize=(20,20))
cax = ax.matshow(sim_matrix, interpolation='nearest')
ax.grid(True)
plt.title('Film Similarity matrix')
plt.xticks(range(len(sub_df)), ID_vect, rotation=90, fontsize=10)
plt.yticks(range(len(sub_df)), ID_vect, fontsize=10)
fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1])
plt.show()

----
### Analysis similarity metric

-> étudier déplacement des clusters en fonction permet de voir quels films influencent quel films? 
- quels pays influencent quel pays?
- quels genres influencent quels genres?
- quels topics apparaissent disparaissent? trends dans le cinéma? 

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
cax = ax.matshow(sim_dict['genre'][0:100,0:100], interpolation='nearest')
ax.grid(True)
plt.title('Film Similarity matrix for one feature')
#plt.xticks(range(len(sub_df)), ID_vect[0:20], rotation=90)
#plt.yticks(range(len(sub_df)), ID_vect[0:20])
fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1])
plt.show()

In [ ]:
#dataset for analysis testing (dataset of 1000 films)
df_test_1000=sub_df.copy()
sim_dict_test_1000, ID_vect_1000 = [sim_dict, ID_vect] 
features=['genre','language','countries','date','actors','characters','director','color','runtime','headlines']

sim_feature=sim_dict_test_1000['genre']

print(type(sim_feature))
print(type(ID_vect_1000))
print(np.shape(sim_feature))
print(np.shape(df_test_1000['genres']))
df_test_1000.head(2)

In [ ]:
#Preprocess df and associated similarity matrix to remove movies that don't have a release date
index = np.where(df_test_1000['release_date'].isna())[0]
df_test_1000=df_test_1000[df_test_1000['release_date'].notna()].reset_index(drop=True)
sim_feature=np.delete(sim_feature,index,0)
sim_feature=np.delete(sim_feature,index,1)

print(np.shape(sim_feature))
print(np.shape(df_test_1000))
df_test_1000.head()

In [ ]:
#algo clustering sur similarité
print(df_clean['release_date'].min())
print(df_clean['release_date'].max())
print(df_test_1000['release_date'].min())
print(df_test_1000['release_date'].max())
df_test_1000['decade'] = df_test_1000.copy()['release_date'].apply(lambda x: x-x%10)
df_test_1000.groupby(df_test_1000['release_date']).count()['wikiID'].plot()
plt.show()
df_test_1000.groupby(df_test_1000['decade']).count()['wikiID'].plot()
plt.show()

In [ ]:
#for each decade show similarity
df_decades=df_test_1000.groupby(df_test_1000['decade'])

In [ ]:
a=0
decade_similarity={}            #each key is a decade (ex: 1940-1949 or 1990-1999)
decade_similarity_additive={}   #each key is from the beginning until a decade (ex: 1890-1849 or 1890-1889)
filter_index_tot=[]

for decade, group in df_decades:
    filter_index=group.index.tolist()
    filter_index.sort()
    decade_similarity[str(int(decade))+'-'+str(int(decade+9))]=sim_feature[np.ix_(filter_index,filter_index)]
    filter_index_tot.extend(filter_index)
    filter_index_tot.sort()
    decade_similarity_additive['until '+str(int(decade+9))]=sim_feature[np.ix_(filter_index_tot,filter_index_tot)]
    """
    #print(filter_index_tot)
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.matshow(decade_similarity[str(int(decade))+'-'+str(int(decade+9))][0:100,0:100], interpolation='nearest')
    ax.grid(True)
    plt.title('Film Similarity matrix for one feature')
    #plt.xticks(range(len(sub_df)), ID_vect[0:20], rotation=90)
    #plt.yticks(range(len(sub_df)), ID_vect[0:20])
    fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1])
    plt.show()
    """
    """
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.matshow(decade_similarity_additive['until '+str(int(decade+9))][0:100,0:100], interpolation='nearest')
    ax.grid(True)
    plt.title('Film Similarity matrix for one feature')
    #plt.xticks(range(len(sub_df)), ID_vect[0:20], rotation=90)
    #plt.yticks(range(len(sub_df)), ID_vect[0:20])
    fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1])
    plt.show()
    """

print(decade_similarity.keys())
print(decade_similarity_additive.keys())
#print(decade_similarity.values())

--- 
### 3. Topic extraction method
To compare plots, we will use a topic extraction algorithm. There are a lot that exist, but we will concentrate on three possible: LDA, Doc2Vec and BERTopic.

#### Preprocessing method on a document corpus before topic extraction

Here is a preprocessing approach that can be implemented before using a topic extraction analysis
1. Removing the stop words from the documents which are the most common words occuring in texts that give no additional concept. It can be done with `Java` using `MySQL`.
2. Removing the numbers appart from years, the non-asci characters, and most common occuring names (ex: James, Robert, John)
3. Handle pural and singular form of the same word by lemmatizing.
4. We can filter the words using tf-idf. We can compute the tf-idf for each word of each plot and keep the words with highest tf-idf score. The threshold for a word to be kept has to be defined.

#### LDA: Latent Dirichlet Allocation

<div>
<img src="images/LDA_example.png" width="800"/>
</div>

**Definition**

LDA is an algorithm that can be used for topic extraction in texts. It is an unsupervised machine-learning model that takes documents as input and finds topics as output. A topic is represented as a weighted list of words. The model also says in what percentage each document talks about each topic. 

**Implementation**

LDA can be implemented using the `GenSim` library. When using LDA to analyse topics in a corpus, it needs some preprocessing steps before applying the algorithm to make it more efficient. We detailed one preprocessing approach above.


After this processing we can apply LDA on our database by tuning some parameters:
<br>
$K$: the number of topics we look for
<br>
$\alpha$: K-dimension vector of positive reals that represent the prior weights of topic K in a document which affects the document-topic distribution. 
<br>
$\eta$: V-dimension vector of positive reals that represents the prior weights of each words in topics which affects the topic-word distribution

If we chose a symetric LDA, the weights $\alpha$ would be the same for all topics and the weights $\eta$ would be the same for all words in a topic. The smaller the $\alpha$ the fewer topics per document, the fewer the $\eta$ the fewer words per topic.

#### Doc2Vec topic extraction
**Definition**

Doc2Vec is an unsupervised algorithm that learns fixed-length feature vectors for paragraphs/documents/texts. Then we can compare these vectors to assess the similarity between documents. Doc2vec allows to generate a semantic space which is a spatial space where distance among vectors are indicator of semantic similarity. This semantic space consisting of word and document vectors is a continuous representation of topics, unlike LDA where topics are sampled from a discrete space. It means that the dense areas having high concentration of document can be thought of having similar topics and can be best represented by nearby embedded words.

**Implementation**

It can be implemented using the `GenSim` library with the class `Doc2Vec` that extends the class `Word2Vec`. 

#### BERTopic

<div>
<img src="images/BERTopic.png" width="600"/>
</div>

**Definition**
BERTopic is a guided semi-supervised topic modeling algorithm. Contrary to LDA, it uses embeddings (thus semi-supervised) and class-based tf-idf (ctf-idf) to find easily interpretable topics and makes it more stable to small variations. It works in three stages:

- Embed the documents
- Cluster the documents
- Create a list of topics and their representation

With this algorithm it is possible to find a list of topics and also give the probability of each topic in each document.

**Implementation**

BERTopic can be implemented using the python `BerTopic` library. As with LDA, we can preprocess the documents (plots in our case) before giving it to the BERTopic for better results.

Each stage of the BERTopic can be parameterized to get better results:
<br>
$Embedding$: by default it uses the `paraphrase-MiniLM-L6-v2` sentence transformers, but we can use any other embedding technique.
<br>
$Clustering$: by default it uses HDBSCAN (hierarchical DBSCAN) to cluster the documents, but to apply this efficiently UMAP is used to reduce the dimensionality of the embeddings. It is possible to change the dimension reduction algorithm and the clustering algorithm, for example we could import K-means form `sklearn.cluster` and use that.
<br>
$Topic$ $representation$: by default it uses class-based tf-idf, meaning it adjusts the tf-idf inside each class (cluster) in relation with the frequency of a word inside the class. Here again it can be tuned for example to try to show different representatives of a cluster. This can be useful to avoid having "bycycle", "cycling" and "bike" as the representatives of a cluster, where only one of them would have been enough.

#### Computation of similarity from topic extraction

Regardless of the method of analysis of the plot, we can have a vector representation at the end:
- Doc2Vec gives a vector directly
- LDA gives the freqency of each topic per document. We can build a vector for each document where each element represents a topic, and it will be proportional to the frequency of the topic.
- BERTopic gives a probability of each topic per document. Similarly to LDA we can build a vector where each element is a topic and fill it with the corresponding probability.

We can then compare these vectors using the cosine similarity for example.

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

#convert to list 
docs = df_clean["plot"].to_list()

#stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

# Prepare embeddings
sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(docs, show_progress_bar = True)
print("embedded")


# Create topic model
model = BERTopic(vectorizer_model=vectorizer_model, verbose = True, calculate_probabilities=True, nr_topics=50)
print("model created")
topics, probabilities = model.fit_transform(docs, embeddings)

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# create model 
 
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

model = BERTopic(vectorizer_model=vectorizer_model, verbose=True, calculate_probabilities=True)
 
#convert to list 
docs = df_clean["plot"].to_list()

In [ ]:
topics, probabilities = model.fit_transform(docs)

In [ ]:
model.get_topic_freq()

#df_plot_topics = df_plot.copy()
#df_plot_topics = df_plot_topics.drop(["plot"], axis=1)

#df_plot_topics = pd.DataFrame(model.probabilities_)
#print(model.probabilities_.shape)
#df_plot_topics["wikiID"] = df_plot["wikiID"]

#df_plot_topics["topics"] = np.zeros(model.probabilities_.shape[0])
#df_plot_topics["topics"] = df_plot["topics"].astype('object')

#for i in range(model.probabilities_.shape[0]):
#    df_plot_topics.at[i,"topics"] = model.probabilities_[i]
#df_plot_topics
#model.probabilities_.tofile("model_probs")
embeddings.shape

In [ ]:
model.visualize_distribution(probabilities[0], min_probability = 0.0, height = 2000)

In [ ]:
model.get_topic_freq()

In [ ]:
model.save("bertopic_model_new_embedding")

In [ ]:
loaded_model = BERTopic.load("bertopic_model")

In [ ]:
loaded_model.get_topic_freq()

In [ ]:
model.visualize_documents(docs, embeddings = embeddings)

In [ ]:
model.visualize_term_rank()

In [ ]:
model.visualize_barchart()

In [1]:
import numpy as np
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

model_probabilities = np.fromfile("model_probs", dtype = 'float')
length = model_probabilities.shape[0]
model_probabilities = model_probabilities.reshape(37476, int(length/37476))
print(model_probabilities.shape)

In [ ]:
#list_vectors = model_probabilities
list_vectors = embeddings
print("copied")
print(list_vectors.shape)

In [ ]:
plots_similarity = sklearn.metrics.pairwise.cosine_similarity(list_vectors)
print("cos calculated")
plots_similarity *= 65535
print("normalized")
plots_similarity2 = plots_similarity.astype('uint16', copy=False)
print("converted", plots_similarity2.dtype)
plots_similarity2.tofile("plots_similarity")
print("saved")

In [ ]:
plots_similarity2 = np.fromfile("plots_similarity", dtype = 'uint16')

In [ ]:
plots_similarity2.shape

In [ ]:
upper_sim = np.triu(plots_similarity2, 1)
threshold_sim = 50000
largest_edges = upper_sim*(upper_sim>=threshold_sim)
indices = np.transpose(np.nonzero(largest_edges))

i = 0
print(len(indices))
for test_idx in range(len(indices)):
    #if(len(df_clean["characters"].iloc[indices[test_idx][0]]) >= 3 and 
    #   len(df_clean["characters"].iloc[indices[test_idx][1]]) >= 3 ):
    print(indices[test_idx])
    print(df_clean["name"].iloc[indices[test_idx][0]], df_clean["release_date"].iloc[indices[test_idx][0]], df_clean["wikiID"].iloc[indices[test_idx][0]])
    print(df_clean["name"].iloc[indices[test_idx][1]], df_clean["release_date"].iloc[indices[test_idx][1]], df_clean["wikiID"].iloc[indices[test_idx][1]])
    i += 1
print(i)

In [ ]:
upper_sim[8364][25341]
#df_clean["plot"].iloc[87]

## Full graph

In [2]:
a = np.fromfile("../similarities/jacc_genre", dtype = 'uint16')
a = a.reshape(37476, 37476)
print("loaded genre")
b = np.zeros((37476,37476)).astype('uint16')
b += (0.45*a).astype('uint16')
print("genre in b")
a = np.fromfile("../similarities/plots_similarity_embeddings", dtype = 'uint16')
a = a.reshape(37476, 37476)
print("loaded plot")
b += (0.45*a).astype('uint16')
print("plot in b")
a = np.fromfile("../similarities/cosine_headlines", dtype = 'uint16')
a = a.reshape(37476, 37476)
print("loaded headline")
b += (0.1*a).astype('uint16')
print("headline in b")

loaded genre
genre in b
loaded plot
plot in b
loaded headline
headline in b


In [ ]:
import networkit as nk

#remove diag and lower triangle
upper_sim = np.triu(b, 1)

print("calculated upper tri")

#keep above threshold
threshold_sim = 45000
largest_edges = upper_sim*(upper_sim>=threshold_sim)

print("selected largest edges")

#find indices of largest values
indices = np.transpose(np.nonzero(largest_edges))

print("nb_edges:", len(indices))

matrix_size = b.shape[0]
G = nk.Graph(matrix_size, weighted = True)

print("created graph nodes")

for i in range(len(indices)):
    G.addEdge(indices[i][0], indices[i][1], b[indices[i][0]][indices[i][1]])

print("created graph edges")

In [ ]:
nk.overview(G)

In [ ]:
communities = nk.community.detectCommunities(G)

In [ ]:
scc = nk.components.StronglyConnectedComponents(G)
scc.run()
print("number of components: ", scc.numberOfComponents())

In [ ]:
list_weights = []
list_edges = []
for u, v, w in G.iterEdgesWeights():
    if w > 60000:
        list_weights.append(w)
        list_edges.append([u,v])
#for i range G.degree(1)
print(len(list_edges))

In [ ]:
print(list_edges)

In [ ]:
print(len(list_edges))
for test_idx in range(len(list_edges)):
    #if(len(df_clean["characters"].iloc[indices[test_idx][0]]) >= 3 and 
    #   len(df_clean["characters"].iloc[indices[test_idx][1]]) >= 3 ):
    print()
    print(list_weights[test_idx])
    #print(list_edges[test_idx], list_weights[test_idx])
    print(df_clean["name"].iloc[list_edges[test_idx][0]], df_clean["release_date"].iloc[list_edges[test_idx][0]], df_clean["wikiID"].iloc[list_edges[test_idx][0]])
    print(df_clean["name"].iloc[list_edges[test_idx][1]], df_clean["release_date"].iloc[list_edges[test_idx][1]], df_clean["wikiID"].iloc[list_edges[test_idx][1]])

In [ ]:
test_idx = 8
print(df_clean["name"].iloc[list_edges[test_idx][0]], df_clean["release_date"].iloc[list_edges[test_idx][0]], df_clean["wikiID"].iloc[list_edges[test_idx][0]])
print(df_clean["plot"].iloc[list_edges[test_idx][0]])
print(df_clean["name"].iloc[list_edges[test_idx][1]], df_clean["release_date"].iloc[list_edges[test_idx][1]], df_clean["wikiID"].iloc[list_edges[test_idx][1]])
print(df_clean["plot"].iloc[list_edges[test_idx][1]])

### 4. Tuning the weights using sequels

How to tell if our similarity function between two movies is working well? First of all, we can't say it for sure, since it is a subjective question. However usually we can still agree on these points:

* Movies that are part of a sequel should have a high similarity
* Movies with completely different genres should have a low similarity (for exmaple one is a "Adventure/Aciton" movie and the other a "Romantic Comedy")

Hence we can build two datasets, one of pairs of movies we expect to have a high similarity and the other of pairs of movies we expect to have low similarity. We can then use these datasets to assess if our similarity function is giving the "right" values.

**Movies from sequels**

Before creating pairs of movies that are similar we should group movies by sequels. Unfortunately, we don't have this data directly accessible in our dataset, but by using the name.clusters.txt file we can have characters that are re-used. It doesn't necesseraly mean that the movie is a sequel (for example with Sherlock Holmes it can be the another representation of him playing), but we still expect these movies to be similar.

First, using the `df_char` dataframe from character.metadata.tsv and the `freeID_char_map` given for each character, we find the unique `WikiID` movie in which the character is played.

In [ ]:
df_char_names["WikiID"] = [ df_char[df_char['freeID_char_map'] == x]["WikiID"].item() for x in df_char_names["freeID_char_map"]]

Then we group all characters together, so for each character we have a list of movies in which he played (represented by their WikiID).

In [ ]:
df_char_grouped = df_char_names.groupby("char_name")["WikiID"].apply(set).to_frame()
df_char_grouped.info()

We could stop here, but right now we still have repetitions: for example Harry Potter and Hermione Granger will appear in the same movies, it would be good to group them together to have a list of separate sequels. However we will have a problem: what to do if only some actors repeat? For example if we take "The Lord of the Rings" and "The Hobbit", Gandalf will appear in both, Aragorn only in "The Lord of the Rings" and Thorin Oakenshield only in "The Hobbit". In our case, we will assume if some characters repeat in these sequels and others don't, it's still highly likely very similar movies, so we will group them all together.

In other words, we group the movies together until we have disjoint movie sets.

Because the list of grouped characters isn't that big (970 rows), we can work with python lists instead of the panda dataframe. We create a list of movie sets (each element of the list comes from one character):

In [ ]:
starting_list = []
# Iterate over each row
for index, rows in df_char_grouped.iterrows():
    # append the list to the final list
    starting_list.append(rows["WikiID"])

Then we join the movies sets from different characters if they both appear in at least one same movie. To do it, we create an empty list, and then iteratively check if we should append a new set (if it has no intersection with the previously added) or make a union with an already existing set (if there is an intersection). To assure that the sets are completely disjoint we run this merging algorithm until the starting "individual" list has the same lenght as the "joined" list.

In [ ]:
joined_characters = []
same = False

while not(same):    
    for i in starting_list:
        joined = False
        for j in range(len(joined_characters)):
            if joined_characters[j] & i: #there is an intersection in the sets
                joined_characters[j] = joined_characters[j] | i #the union of both sets
                joined = True
        if joined == False:
            joined_characters.append(i)
    
    if len(starting_list) == len(joined_characters):
        same = True
    else:
        starting_list = joined_characters
        joined_characters = []

print(len(joined_characters))

We get 583 different sequels, which will be enough to test for similarities. To actually compute a similarity we will need to choose a sequel, and then choose a pair of movies in this sequel. If a sequel has n movies in it, the number of different pairs we can build is n*(n-1)/2. If we compute this for all sequels, we find that we can have 10929 different possible pairs:

In [ ]:
sum_pairs = 0
for i in joined_characters:
    n = len(i)
    sum_pairs = sum_pairs + n*(n-1)/2

print(sum_pairs, "different possible pairs")

**Movies with different genres**

For this milestone we didn't built a dataset of pairs of movies with different genres, because we're confident it won't be a problem to find them. Building a dataset also won't be a problem: you sample a movie randomly, and then sample another one to form a pair. If the second one has an intersection in the genres, you resample it, until you find one with 0 intersection. We won't create all possible pairs because the dataset will be too large.

**Conclusion**

With this two datasets we will be able to test our similarity function, and possibly adjust its values to have a high similarity in sequels and low similarity in movies from completely different genres. We will also need to be careful to not base the tuning completely on this as by the construction of the datasets it will give a high importance to similar characters and genres.

---
## Visualization method

We want to be able to visualize our graph. But how do you represent a graph? You could start by placing one node, and then place its neighbours at a distance (or 1/similarity) around him, then place their neighbours and so on. But with that method you can have a problem of not respecting distances with already 3 nodes A, B and C: for example the distance AB = BC = 1 and distance AC = 10. It means we probably won't be able to represent the graph perfectly, but we can still try to do the best possible by plotting nodes with a high similarity close to each other and those with a low similarity far from each other. This is done using Graph Layout Algorithms.

### Gephi

<div>
<img src="images/gephi.jpg" width="600"/>
</div>

**Definition**

Gephi is an "open-source platform for visualizing and manipulating large graphs". Their Graph Layout Algorithm is force based, meaning nodes will repell and attract each other in function of the distance between them, and the visualization is in the equilibrium state.
 

**Implementation**

We found a [github repository](https://github.com/la-rana-kermit/Gephi-python-module) to interface Gephi with Python, which is what we'll try to use to implement. However this repository doesn't seem very active, so if we get some problems we might switch to the Gephi Software directly.


---

## Annexe: Mathematical definitions related to similarity

#### 1.1 Cosine similarity
$$
cosine \: similarity(A,B)=S_c(A,B)=cos(\theta)=\frac{A \cdot B}{\|A\| \|B\|}=\frac{\sum_{i=1}^{n}A_iB_i}{\sqrt{\sum_{i=1}^{n} A_i^2}\sqrt{\sum_{i=1}^{n} B_i^2}}
$$
Where $A,B \in \mathbb{R}^n$, $S_c(A,B) \in [-1,1]$ where -1 means that the two vectors are exactly opposite, and 1 means that they are exactly similar and 0 means that they are orthognonal which shows decorrelation.

#### 1.2 Centered Cosine similarity
$$
centered \: cosine \: similarity(A,B)=\frac{(A-\overline{A}) \cdot (B-\overline{B})}{\|A-\overline{A}\| \|B-\overline{B}\|}
$$
Where A and B have been normalized before by substracting their mean.

#### 1.3 Soft cosine similarity
$$
soft \: cosine \: similarity(A,B)=\frac{\sum_{i,j}^{n}s_{ij}A_iB_j}{\sqrt{\sum_{i,j}^{n} s_{ij}A_iA_j}\sqrt{\sum_{i,j}^{n} s_{ij}B_iB_j}}
$$
where $s_{ij}$=similarity($feature_i$,$feature_j$). For example if $s_{ii}$=1 and $s_{ij}$=0 $\forall i\neq j$ then there is no similarity between features, then the soft cosine similarity is equal to the cosine similarity. In the case where features are words, the matrix $S$ has to define the similarity between words.

#### 1.4 Associated distance with cosine similarity
- **Angular distance**

if $A_i,B_i \in \mathbb{R}$ 
$$
angular \: distance=D_\theta=\frac{arccos(S_c(A,B))}{\pi}=\frac{\theta}{\pi}
$$
$$
angular \: similarity=S_\theta=1-D_\theta=1-\frac{\theta}{\pi}
$$
if $A_i,B_i \in \mathbb{R}$ and $A_i,B_i\geq 0$
$$
angular \: distance=D_\theta=\frac{2 \cdot \textrm{arccos}(S_c(A,B))}{\pi}=\frac{2\theta}{\pi}
$$
$$
angular \: similarity=S_\theta=1-D_\theta=1-\frac{2\theta}{\pi}
$$
Where the angular distacne is a formal distance metric, however the arccos computation cost makes it more computationally expensive and slower.

- **Cosine distance**

$$
cosine \: distance=D_c=1-S_c(A,B)
$$
Where the cosine distance is an unformal distance metric (it does not respect the triangle inequality or Schwarz inequality) but it is less computationally expensive.


- **L2-normalized Euclidean distance**

From the L2 distance defined as followed: $ \|x\|_2=\sqrt{\sum x_i^2}=\sqrt{x.x} $ and the euclidean distance defined as followed: 
$ d(A,B)=|A-B|=\sqrt{\sum _{i=1}^{n} (A_i-B_i)^2 } $, we get the L2-normalized Euclidean distance:
$$
L2-normalized \: Euclidean \: distance=\sqrt{\sum _{i=1}^{n} (A_i'-B_i')^2} \quad \textrm{where} \quad A'=\frac{A}{\|A\|_2}
$$

The cosine similarity and associated distances reflects relative rather than absolute comparison of vectors. For example vectors $A$ and $\alpha A$ where $\alpha \in \mathbb{R}$ are maximally similar. Therefore this similarity is appropriate for data where frequency is more important than absolute value. For text comparison it can be very useful, we could compare the frequency of terms in a document.  

#### 1.5 Jaccard similarity coefficient

$$
J(A,B) = \frac{|A \cap B|}{|A \cup B|} = \frac{|A \cap B|}{|A| + |B| - |A \cap B|}
$$
where $J(A,B) \in [0,1]$.
It measures similarity between finite sample sets, and is defined as the size of the intersection divided by the size of the union of the sample sets.  

#### 1.6 Associated distance with Jaccard similarity
$$
d_J(A,B)=1-J(A,B)=\frac{|A\cup B|-|A\cap B|}{|A\cup B|}
$$
The Jaccard distance, which measures dissimilarity between sample sets, is complementary to the Jaccard coefficient and is obtained by subtracting the Jaccard coefficient from 1, or, equivalently, by dividing the difference of the sizes of the union and the intersection of two sets by the size of the union.